In [ ]:
from textwrap import dedent
from openai import OpenAI
from prompt_optimizer import PipelineOptimizer, llm_node, OpenAIAdapter

# Basic steps:
# 1) Add a `llm_node` decorator to an LLM function
# 2) Create an evaluation template
# 3) Define your "training" data
# 4) Set up the optimizer
# 5) Check the results!

client = OpenAI()

system_prompt = "you are a helpful assistent"

# Add a `llm_node` decorator around a function which takes a prompt to be optimised
# the input should be a component to be optimised (`system_prompt`` in this case)
@llm_node(system_prompt=system_prompt)
def answer_query(query: str, system_prompt: str = system_prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
    )
    return response.choices[0].message.content.strip()

# Define a template for evaluating the final answer
# you may use {pipeline_output} and any other custom key word arguments (kwargs)
exact_match_evaluator = dedent(
    """
    Your task is to judge the quality if the response given by an AI assistent.
    Below are the <question>, <assistent_response> and <expected_answer>.
    A correct answer would be an exact string match between the <assistent_response> and the
    <expected_answer>

    <question>{query}</question>
    <assistent_response>{pipeline_ouput}</assistent_response>
    <expected_answer>{golden_answer}</expected_answer>
    """
)

# Define your dataset, this could have any number of items in the dictionary
# and contain any number of examples
data = [{
    "query": "what is the capital of France?",
    "golden_answer": "<answer>Paris</answer>"
}]

# Instantiate an optimizer and pass in an `LLMCallable`
# (in this case using the OpenAIAdapter)
llm = OpenAIAdapter(client)
optimizer = PipelineOptimizer(llm)

# Optimize
optimizer.optimize(
    iterations=1,
    pipeline_func=answer_query,
    evaluation_template=exact_match_evaluator,
    data=data,
)

# Check the results
result = answer_query("what is the capital of England?").value

print(result)
# ---- expected answer: ----
# <answer>London</answer>

print(optimizer.get_prompt_info())
# ---- expected answer: ----
# Node: `answer_query`
#  system_prompt: Provide the answer strictly in the required format, enclosing your response within <answer></answer> tags. Ensure that the content is clear, concise, and directly addresses the question while maintaining a helpful tone.